In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [3]:
import pandas as pd
import numpy as np
from omnixai.data.tabular import Tabular
from omnixai.explainers.data import DataAnalyzer
from omnixai.preprocessing.base import Identity
from omnixai.preprocessing.encode import LabelEncoder
from omnixai.preprocessing.tabular import TabularTransform
from omnixai.explainers.tabular import TabularExplainer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,r2_score

In [4]:
data=pd.read_csv('smsspamcollection.csv',sep='\t')
data=data.dropna()
data['label'] = data['label'].astype(object)

In [5]:
labs=[]
for i,j in zip(data.label,range(len(data.label))):
  if(i=='ham'):

    labs.append(0)
  else:
    labs.append(1)


In [6]:
data['label']=labs

In [7]:
data['label'] = data['label'].astype(int)

In [8]:
x=data.iloc[:,1].values
y=data.iloc[:,0].values

In [9]:
# data=data.iloc[:,1:]
# target=data.iloc[:,0]
data.columns

Index(['label', 'message', 'length', 'punct'], dtype='object')

In [10]:
tabular_data = Tabular(
    data,
    feature_columns=data.columns,
    categorical_columns=[ 'message'],
    target_column='label'
)

In [11]:
explainer = DataAnalyzer(
    explainers=["correlation", "mutual", "chi2"],
    data=tabular_data,
    mode =" classification"
)
explanations = explainer.explain_global()


In [12]:
transformer = TabularTransform(
    target_transform=Identity()
).fit(tabular_data)

In [13]:
from omnixai.preprocessing.tabular import TabularTransform
transformer = TabularTransform().fit(tabular_data)
class_names = transformer.class_names
x = transformer.transform(tabular_data)

In [14]:
train, test, train_labels, test_labels =train_test_split(x[:, :-1], x[:, -1], train_size=0.80)

In [15]:
# from sklearn.model_selection import train_test_split

# x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.25)

In [16]:

# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer()
# x_train_vocab=count_vect.fit_transform(x_train)

In [17]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf=TfidfVectorizer()

# # from sklearn.feature_extraction.text import TfidfTransformer
# # tfidf=TfidfTransformer()
# x_train_tfidf=tfidf.fit_transform(train)
from sklearn.linear_model import SGDClassifier

# Create the model
clf = SGDClassifier(loss='log', random_state=42)


clf.fit(train,train_labels)
# y=np.array(y,dtype=np.float32)
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline

# pipeline = Pipeline([
#     ('vect', TfidfVectorizer()),
#     ('clf', LogisticRegression())
# ])

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline

# # Define the pipeline
# pipeline = Pipeline([
#     ('vect', TfidfVectorizer()),
#     ('clf', LogisticRegression())
# ])

# pipeline.fit(x_train, y_train)

SGDClassifier(loss='log', random_state=42)

In [18]:

train_data = transformer.invert(train)
test_data = transformer.invert(test)

In [19]:
preprocess = lambda z: transformer.transform(z)

In [20]:
from omnixai.explainers.tabular import TabularExplainer
from omnixai.visualization.dashboard import Dashboard

explainers = TabularExplainer(
    explainers=["lime", "shap", "pdp"],
    mode="regression",
    data=train_data,
    model=clf,
    preprocess=preprocess,
    params={
        "lime": {"kernel_width": 3},
        "shap": {"nsamples": 100}
    }
)

In [ ]:
test_instances = transformer.invert(test[0:5])
local_explanations = explainers.explain(X=test_instances)
global_explanations = explainers.explain_global()

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from omnixai.explainers.prediction import PredictionAnalyzer

analyzer = PredictionAnalyzer(
    mode="classification",
    test_data=test_data,                           # The test dataset (a `Tabular` instance)
    test_targets=test_labels,                      # The test labels (a numpy array)
    model=clf,                                   # The ML model
    preprocess=lambda z: transformer.transform(z)  # Converts raw features into the model inputs
)
prediction_explanations = analyzer.explain()

In [ ]:
dashboard = Dashboard(
    instances=test_instances,                        # The instances to explain
    local_explanations=local_explanations,           # Set the generated local explanations
    global_explanations=global_explanations,         # Set the generated global explanations
    class_names=class_names,                         # Set class names
)
dashboard.show()                                     # Launch the dashboard

# New section